# Building a Database for Crime Reports in Boston

In this project, we will create a PostgreSQL database named `crimes_db` to store and manage crime data from Boston. Using the provided `boston.csv` dataset, we will:

- Design a schema (`crimes`) and a table (`boston_crimes`) with appropriate datatypes.
- Import the data into the database.
- Create user roles (`readonly` and `readwrite`) with appropriate privileges.
- Assign users to these roles to manage database access.

## Creating the Crime Database

We will start by creating a database for storing our crime data, along with a schema for the tables. 

In [ ]:
import psycopg2

conn = psycopg2.connect(dbname="postgres", user="postgres")
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE DATABASE crime_db;")
conn.close()

conn = psycopg2.connect(dbname="crime_db", user="postgres")
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes;")

## Obtaining the Column Names and Sample

Before we start creating tables, let's gather some data about our crime dataset so that we can select the right datatypes to use.

In [ ]:
import csv

with open("boston.csv") as f:
    reader = csv.reader(f)
    col_headers = next(reader)
    first_row = next(reader)

print("Column names -->", col_headers)
print("First row of data -->", first_row)

## Creating an Auxiliary Function

To help us identify the right datatypes for the columns, we'll create a function that computes a Python set with all of the distinct values contained within a column.

The function will be useful for:

- Checking whether an enumerated datatype might be a good choice for representing a column.
- Computing the maximum length of any text-like column to select appropriate sizes for `VARCHAR` columns. 

In [ ]:
def get_col_set(csv_filename, col_index):
    with open(csv_filename) as f:
        reader = csv.reader(f)
        next(reader)
        return set(row[col_index] for row in reader)

for i in range(len(col_headers)):
    num_unique_values = len(get_col_set("boston.csv", i))
    print(f"- Column '{col_headers[i]}' has {num_unique_values} unique values")

## Calculating the Maximum Length of Values in the Description Column

In [ ]:
print(col_headers)

In [ ]:
desc_unique_values = get_col_set("boston.csv", 2)
desc_max_length = max(len(val) for val in desc_unique_values)

print(f"The maximum length of values contained within the 'description' column is {desc_max_length}")

## Choosing Datatypes

While `incident_number` and `offense_code` column values are represented as strings in the CSV, they are clearly numeric identifiers. As such, we will store the values as an `INTEGER` type.

From our calculation above, we found that the maximum length of values contained within the `description` column is 58. To provide some additional flexibility, we will store these values as `VARCHAR(100)`.

For the `date` column, we will use the corresponding `DATE` datatype.

Latitude ranges from -90.000000 to 90.000000 and Longitude ranges from -180.000000 to 180.000000. Using these ranges, we will store the `lat` and `long` column values using the `DECIMAL` datatype. We can make use of `Precision` (total number of digits) and `Scale` (number of digits after the decimal point) to specify `DECIMAL(9, 6)`, in line with the typical ranges mentioned above.

As the `day_of_the_week` column only has 7 unique values, it is a good candidate for using an enumerated datatype. We'll create this datatype before creating our table.

In [ ]:
cur.execute("CREATE TYPE day_of_week AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');")

## Creating the Table

Now that we have identified suitable datatypes for our crime data, we can go ahead and create our table.

In [ ]:
cur.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_number integer PRIMARY KEY,
        offense_code integer,
        description varchar(100),
        date date,
        day_of_the_week day_of_week,
        lat decimal(9, 6),
        long decimal(9, 6)
    );
""")

## Loading the Data

Now that we have created our table, we can load the data into it.

In [ ]:
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)

We can then query our table to view the first few rows.

In [ ]:
cur.execute("SELECT * FROM crimes.boston_crimes LIMIT 5")
first_five = cur.fetchall()

for row in first_five:
    print(row)